# Grupo 03 - Desafío 01
## Estrategia de construcción
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode

### Importación de librerías

In [ ]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

# Carga e inicialización

In [ ]:
# Definición de df que permite registrar los valores recuperados de cada columna
# Por el momento se encuentra bajo desarrollo

df_recup = pd.DataFrame()

In [ ]:
# Definición de filas a cargar en las pruebas

ca_filas = 40

In [ ]:
# Carga del dataset en un df

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)

In [ ]:
# Reemplado de NaN's

df = g3.reemplaza_nan('sin datos',df)

In [ ]:
# Verificación del reemplazo

df.shape()

## Armado inicial de df post-produccion
* Este bloque se reserva para realizar la precarga inicial dentro de un df de post-producción que finalmente se transformará en dataset de producción.

In [ ]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

## Procesamiento de limpieza y carga de columnas depuradas en df post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza mas abajo dentro del título **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora.



In [ ]:
# Depuración de expresion 'ambientes' en 'description' y generacion de columna en pospro

def recuperar_ambientes(pospo):
    pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    df_temp = g3.limpiar_columna_x_clave(dic, df1)
    pospo = g3.agregar_columna('ambiente', pospo, df_temp)
    return pospo


In [ ]:
pospo = recuperar_ambientes(pospo)
pospo.shape

In [ ]:
# Depuración 'precio usd' en 'description' y agregado de columna en pospro

def recuperar_usd(pospo):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'usd') # Genera DF acotado con los valores recuperados unicamente
    pospo = pd.merge(pospo, df2,on='indice', how='left')
    return pospo


In [ ]:
pospo=recuperar_usd(pospo)
pospo.shape


In [ ]:
def recuperar_metros(pospo):
    pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado_simple(df1, 'metros2') # Genera DF acotado con los valores recuperados unicamente
    pospo = pd.merge(pospo, df2,on='indice', how='left')
    return pospo

In [ ]:
pospo=recuperar_metros(pospo)
pospo.shape

# Desarrollo